In [12]:
import pandas as pd 
import numpy as np
import statsmodels.stats

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [40]:
aspirin = np.array([1]*104 + [0]*(11037-104) )
placeba = np.array([1]*189 + [0]*(11034-189) )

In [13]:
round(len(aspirin[aspirin==0])/len(aspirin) - \
len(placeba[placeba==0])/len(placeba),4)

0.0077

Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [ ]:
def proportions_confint_diff_ind(sample1, sample2, alpha=0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [65]:
from statsmodels.stats.proportion import proportion_confint
proportion_confint(sum(aspirin), len(aspirin),method='normal')

(0.007620422638288568, 0.011225278186210842)

Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [56]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples,len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries
def odds(data):
    p = data.sum() / data.shape[0]
    return p / (1 - p)

np.random.seed(0)
scores1 = list(map(odds, get_bootstrap_samples(aspirin, 1000)))
scores2 = list(map(odds, get_bootstrap_samples(placeba, 1000)))

print("95% confidence interval",  stat_intervals(scores1, 0.05))
print("95% confidence interval",  stat_intervals(scores2, 0.05))

95% confidence interval [0.00757714 0.0113626 ]
95% confidence interval [0.01499402 0.01996672]


In [57]:
odds = list(map(lambda x: x[1]/x[0], zip(scores1, scores2)))
print("95% confidence interval",  stat_intervals(odds, 0.05))

95% confidence interval [1.44419465 2.34321168]
